In [63]:
from PIL import Image
import matplotlib.pyplot as plt

def filtro_mediana(_img, janela_m=3, janela_n=3):
    img = _img.copy()
    M = img.width
    N = img.height
    m = janela_m//2
    n = janela_n//2
    for i in range(m,M-m):
        for j in range(n,N-n):
            lista = []
            # percorrer mascara
            for x in range(i-m, i+m+1):
                for y in range(j-n, j+n+1):
                    lista.append(_img.getpixel((x,y)))
            lista = sorted(lista)
            pixel = lista[len(lista)//2]
            img.putpixel((i,j), pixel)
                        
    return img


img = Image.open('morfologia.png')
img = filtro_mediana(img,5,5)
img = filtro_mediana(img,3,3)
img = filtro_mediana(img,3,3)
img = filtro_mediana(img,3,3)
img = filtro_mediana(img,3,3)
img = filtro_mediana(img,3,3)
img.save("temp/filtro_mediana.png")

In [64]:
from PIL import Image
import matplotlib.pyplot as plt

def filtro_minima(_img, janela_m=3, janela_n=3):
    img = _img.copy()
    M = img.width
    N = img.height
    m = janela_m//2
    n = janela_n//2
    for i in range(m,M-m):
        for j in range(n,N-n):
            lista = []
            # percorrer mascara
            for x in range(i-m, i+m+1):
                for y in range(j-n, j+n+1):
                    lista.append(_img.getpixel((x,y)))
            lista = sorted(lista)
            pixel_color = lista[0]
            img.putpixel((i,j), pixel_color)
                        
    return img


img = Image.open('morfologia.png')
img = filtro_minima(img,3,3)
img.save("temp/filtro_minima.png")

In [65]:
def filtro_max(_img, janela_m=3, janela_n=3):
    img = _img.copy()
    M = img.width
    N = img.height
    m = janela_m//2
    n = janela_n//2
    for i in range(m,M-m):
        for j in range(n,N-n):
            lista = []
            # percorrer mascara
            for x in range(i-m, i+m+1):
                for y in range(j-n, j+n+1):
                    lista.append(_img.getpixel((x,y)))
            lista = sorted(lista)
            pixel_color = lista[-1]
            img.putpixel((i,j), pixel_color)
                        
    return img


img = Image.open('morfologia.png')
img = filtro_max(img,3,3)
img.save("temp/filtro_max.png")

In [66]:
class ElementoEstruturante:
    def __init__(self, f, ancora=None): 
        '''
        f = matriz contendo o elemento estruturante 
        [
            [x,x,x],
            [x,x,x],
            [x,x,x],
        ]
        ancora = ponto de ancoragem do elemento estruturante. Se vazia o ponto de ancora será o ponto central
        (linha,coluna)
        '''
        self.f = f
        self.m = len(f[0])
        self.n = len(f)
        if ancora:
            self.ancora = {
                'x': ancora[0],
                'y': ancora[1],
            }
        else:
            self.ancora = {
                'x': self.n//2,
                'y': self.m//2,
            }
        self.f_ancora = f[self.ancora['x']][self.ancora['y']]
    
pixel = (0,0,0)
pixel_vermelho = (229,0,28)
pixel_azul = (48,46,192)
pixel_verde = (35,166,59)
pixel_magenta = (144,48,147)


In [67]:

pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,None,None],
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
            [None,None,None],
        ],
        (2,1)
    )
# ponto de ancora linha x coluna



def hit(img, elemento_estruturante, ponto):
    es_m = elemento_estruturante.m # colunas
    es_n = elemento_estruturante.n # linhas
    M = img.width # colunas
    N = img.height # linhas
    x = ponto[1] - elemento_estruturante.ancora['x'] # linha
    for linhas in elemento_estruturante.f:
        y = ponto[0] - elemento_estruturante.ancora['y'] # coluna
        for el in linhas:
            if not el:
                y+=1
                continue
            if x < 0 or y < 0 or x >= N or y >= M:
                return False
            #print(img.getpixel((y,x)), end=",")
            if el != img.getpixel((y,x)):
                return False
            y+=1
        #print('')
        x+=1
    return True


def erosao(_img, elemento_estruturante, pixel):
    img = _img.copy()
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            #print(f"{i}{j}", end=",")
            if hit(_img, elemento_estruturante, (j,i)):
                img.putpixel((j,i), pixel)
            else:
                img.putpixel((j,i), (255,255,255))
            #print(img.getpixel((j,i)), end=",")
        #print('')
        
    return img
                
img = Image.open('teste_3.png').convert('RGB')
img = erosao(img, elemento_estruturante, pixel)
img.save("temp/erosao.png")

In [68]:
pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,None,None],
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
            [None,None,None],
        ],
        (2,1)
    )

# ponto de ancora linha x coluna



def find(img, elemento_estruturante, ponto):
    es_m = elemento_estruturante.m # colunas
    es_n = elemento_estruturante.n # linhas
    M = img.width # colunas
    N = img.height # linhas
    x = ponto[1] - elemento_estruturante.ancora['x'] # linha
    found = []
    for linhas in elemento_estruturante.f:
        y = ponto[0] - elemento_estruturante.ancora['y'] # coluna
        for el in linhas:
            if not el:
                y+=1
                continue
            if x < 0 or y < 0 or x >= N or y >= M:
                y+=1
                continue
            found.append((y,x))
            y+=1
        x+=1
    return found


def dilatacao(_img, elemento_estruturante, pixel):
    img = _img.copy()
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            if _img.getpixel((j,i)) == elemento_estruturante.f_ancora:
                for ponto in find(_img, elemento_estruturante, (j,i)):
                    img.putpixel(ponto, pixel)
        
    return img
                
img = Image.open('teste_1.png').convert('RGB')
img = dilatacao(img, elemento_estruturante, pixel)
img.save("temp/dilatacao.png")

In [69]:
def complemento(_img):
    img = _img.copy()
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            pixel = _img.getpixel((j,i))
            complement = tuple(map(int.__sub__, (255,255,255), pixel))
            img.putpixel((j,i), complement)
    return img

img = Image.open('teste_1.png').convert('RGB')
img = complemento(img)
img.save("temp/complemento.png")


In [70]:
def uniao(_img1,_img2):
    img = _img1.copy()
    if _img1.width != _img2.width or _img1.height != _img2.height:
        raise Exception('As imagens devem ter o mesmo tamanho')
        return
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            pixel1 = _img1.getpixel((j,i))
            pixel2 = _img2.getpixel((j,i))
            img.putpixel((j,i), min(pixel1,pixel2))
    return img

img1 = Image.open('teste_1.png').convert('RGB')
img2 = Image.open('teste_2.png').convert('RGB')
img = uniao(img1, img2)
img.save("temp/uniao.png")

In [131]:
def intersecao(_img1,_img2):
    img = _img1.copy()
    if _img1.width != _img2.width or _img1.height != _img2.height:
        raise Exception('As imagens devem ter o mesmo tamanho')
        return
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            pixel1 = _img1.getpixel((j,i))
            pixel2 = _img2.getpixel((j,i))
            img.putpixel((j,i), max(pixel1,pixel2))
    return img

img1 = Image.open('teste_1.png').convert('RGB')
img2 = Image.open('teste_2.png').convert('RGB')
img = intersecao(img1, img2)
img.save("temp/intersecao.png")

In [132]:
def diferenca(_img1,_img2):
    img = _img1.copy()
    if _img1.width != _img2.width or _img1.height != _img2.height:
        raise Exception('As imagens devem ter o mesmo tamanho')
        return
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            pixel1 = _img1.getpixel((j,i))
            pixel2 = _img2.getpixel((j,i))
            if pixel2 < (255,255,255):
                img.putpixel((j,i), (255,255,255))
    return img

img1 = Image.open('teste_1.png').convert('RGB')
img2 = Image.open('teste_2.png').convert('RGB')
img = diferenca(img1, img2)
img.save("temp/diferenca.png")

In [105]:
pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,None,None],
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
            [None,None,None],
        ],
        (2,1)
    )
# ponto de ancora linha x coluna



def hitRGB(img, elemento_estruturante, ponto):
    M = img.width # colunas
    N = img.height # linhas
    x = ponto[1] - elemento_estruturante.ancora['x'] # linha
    for linhas in elemento_estruturante.f:
        y = ponto[0] - elemento_estruturante.ancora['y'] # coluna
        for el in linhas:
            if not el:
                y+=1
                continue
            if x < 0 or y < 0 or x >= N or y >= M:
                return False
            if not el or img.getpixel((y,x)) == (255,255,255):
                return False
            y+=1
        x+=1
    return True


def erosaoRGB(_img, elemento_estruturante):
    img = _img.copy()
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            if hitRGB(_img, elemento_estruturante, (j,i)):
                img.putpixel((j,i), img.getpixel((j,i)))
            else:
                img.putpixel((j,i), (255,255,255))
    return img
                
img = Image.open('morfologia.png').convert('RGB')
img = erosaoRGB(img, elemento_estruturante)
img.save("temp/erosao.png")

In [106]:
pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
        ],
        (2,1)
    )

def abertura(_img, elemento_estruturante, pixel):
    img = _img.copy()
    img = erosao(img, elemento_estruturante, pixel)
    img = dilatacao(img, elemento_estruturante, pixel)
    return img

img = Image.open('teste_abertura.png').convert('RGB')
img = abertura(img, elemento_estruturante, pixel)
img.save("temp/abertura.png")

In [107]:
pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
        ],
        (2,1)
    )

def fechamento(_img, elemento_estruturante, pixel):
    img = _img.copy()
    img = dilatacao(img, elemento_estruturante, pixel)
    img = erosao(img, elemento_estruturante, pixel)
    return img

img = Image.open('teste_fechamento.png').convert('RGB')
img = fechamento(img, elemento_estruturante, pixel)
img.save("temp/fechamento.png")

In [147]:

pixel_vermelho = (229,0,28)
pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [None,pixel,None],
            [pixel,pixel,pixel],
            [None,pixel,None],
        ],
        (1,1)
    )
print(pixel_vermelho)

def preenche_buraco(_img, elemento_estruturante, pixel, ponto_inicial):
    img = _img.copy()
    new_img = Image.new('RGB',(img.width, img.height), (255,255,255))
    ponto = (ponto_inicial[1], ponto_inicial[0])
    new_img.putpixel(ponto, pixel)
    img_c = complemento(img)
    img_c.show()

    img = dilatacao(new_img, elemento_estruturante, pixel)
    img = intersecao(img, img_c)
    img.show()
    while img != new_img:
        new_img = img.copy()
        img = dilatacao(new_img, elemento_estruturante, pixel)
        img = intersecao(img, img_c)
    return uniao(img, _img)

img = Image.open('teste_preenche_buraco.png').convert('RGB')
img = preenche_buraco(img, elemento_estruturante, pixel_vermelho, (1,1))
img.save("temp/preenche_buraco.png")

(229, 0, 28)


In [142]:
def trocar_cor(_img, pixel_original, pixel_novo):
    img = _img.copy()
    M = img.width
    N = img.height
    for i in range(N):
        for j in range(M):
            if _img.getpixel((j,i)) == pixel_original:
                img.putpixel((j,i), pixel_novo)
            else:
                img.putpixel((j,i), (255,255,255))
    return img
img = Image.open('morfologia.png').convert('RGB')
img = trocar_cor(img, pixel_vermelho, (0,0,0))
#img = trocar_cor(img, (0,0,0), pixel_vermelho)
img.save("questão_02/temp/vermelho.png")

In [261]:

def hit_or_miss(_img, D):
    img = _img.copy()
    m = (D.m+2)
    n = (D.n+2)
    W = [[(0,0,0)]*m for l in range(n)]

    for l in range(n-2)[::-1]:
        for c in range(m-2)[::-1]:
            if D.f[l][c] == None:
                W[l+1][c+1] = None
                continue
            W[l+1][c+1] = tuple(map(int.__sub__, (255,255,255),D.f[l][c]))
            if W[l+1][c+1] == (255,255,255):
                W[l+1][c+1] = None
    
    B1 = D
    B2 = ElementoEstruturante(W)
    img_c = complemento(_img)

    img1 = erosao(img, B1, (0,0,0))
    img2 = erosao(img_c, B2, (0,0,0))
    img = intersecao(img1, img2)
    return img

pixel = (0,0,0)
D = ElementoEstruturante(
        [
            [pixel,pixel,pixel, pixel, pixel],
            [pixel,pixel,pixel, pixel, pixel],
            [pixel,pixel,pixel, pixel, pixel],
            [pixel,pixel,pixel, pixel, pixel],
            [pixel,pixel,pixel, pixel, pixel],
        ],
    )


img = Image.open('teste_1.png').convert('RGB')
img = hit_or_miss(img, D)
img.save("temp/hit-or-miss.png")

In [263]:
def rotaciona(elemento_estruturante):
    es = elemento_estruturante
    elemento = es.f[es.ancora['x']][es.ancora['y']]
    es.f[es.ancora['x']][es.ancora['y']] = 'a'
    new_es = []
    new_ancora = (es.ancora['x'], es.ancora['y'])
    for c in range(es.m):
        linha = []
        for l in range(es.n)[::-1]:
            linha.append(es.f[l][c])
        new_es.append(linha)
    for i in range(len(new_es)):
        for j in range(len(new_es[i])):
            if new_es[i][j] == 'a':
                new_ancora = (i,j)
                new_es[i][j] = elemento
        #print(new_es[i])
    #print(new_ancora)
    return ElementoEstruturante(new_es, new_ancora)
            

pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [(0,0,0),None,None],
            [(0,0,0),(255,255,255),None],
            [(0,0,0),None,None],
        ],
        (1,1)
    )
elemento_estruturante = rotaciona(elemento_estruturante)


In [287]:
def fecho_convexo(_img, elemento_estruturante1):
    img = _img.copy()
    img = erosao(_img, elemento_estruturante, (0,0,0))
    new_img = uniao(_img, img)

    elemento_estruturante2 = rotaciona(elemento_estruturante1)
    elemento_estruturante3 = rotaciona(elemento_estruturante2)
    elemento_estruturante4 = rotaciona(elemento_estruturante3)

    old_img = _img.copy()
    while old_img != new_img:
        old_img = new_img.copy()
        img = erosao(new_img, elemento_estruturante, (0,0,0))
        new_img = uniao(new_img, img)
    return new_img
    

pixel = (0,0,0)
elemento_estruturante = ElementoEstruturante(
        [
            [(0,0,0),None,None],
            [(0,0,0),(255,255,255),None],
            [(0,0,0),None,None],
        ],
        (1,1)
    )

#elemento_estruturante = rotaciona(elemento_estruturante)
img = Image.open('teste_1.png').convert('RGB')
img = fecho_convexo(img, elemento_estruturante)
img.save("temp/fecho_convexo.png")